## Read data

In [ ]:
import pandas as pd

# hits that were "redocked" (placed) for scores and interactions
ref = pd.read_pickle('fragmenstein_hit_replacements_pairs.pkl.gz')
pairs = pd.read_pickle('fragmenstein_placed_pairs.pkl.gz')
trio = pd.read_pickle('fragmenstein_placed_trio.pkl.gz')

In [ ]:
combo = pd.concat([pairs, trio], ignore_index=True)
analogs = combo.loc[combo.outcome == 'acceptable'].copy();
len(ref), len(pairs), len(trio), len(combo), len(analogs)

## Score

In [ ]:
from typing import List
import operator
from rdkit import Chem

hits: List[Chem.Mol] = ref.hit_mols.apply(operator.itemgetter(0)).to_list()

ori_weights = {"N_rotatable_bonds": 1.5,
             "\u2206\u2206G": 1,
             "interaction_uniqueness_metric": -3,
             "N_unconstrained_atoms": 0.2,
             "N_constrained_atoms": -0.1,
             "N_interactions": -2,
             "N_interactions_lost": 2,
             "max_hit_Tanimoto": -0.3,
             "N_PAINS": 7,
             "strain_per_HA": 0.3}


energy_weights = {"N_rotatable_bonds": 1,
                  "\u2206\u2206G": 1,
                  "strain_per_HA": 0.3}

from fragmenstein import Laboratory

Laboratory.score(analogs, ref,
                 suffix='',
                 hits=hits,
                 weights=energy_weights,)

analogs['energy_penalty'] = analogs.ad_hoc_penalty

Laboratory.score(analogs, ref,
                 suffix='',
                 hits=hits,
                 weights=ori_weights,)

analogs['full_penalty'] = analogs.ad_hoc_penalty

analogs = analogs.copy()

## XChem only

Get rid of non-xchem only vcs: in the mergers the native ligand and stolen PDB hits may be present.
We want XChem only mergers

In [ ]:
def has_xchem(names):
    return any([name.replace('A71EV2A-', '')[0] == 'x' for name in names])

analogs['xchem_based'] = analogs.hit_names.apply(has_xchem)
analogs = analogs.loc[analogs.xchem_based]

## Cluster

In [ ]:
# from kmodes.kmodes import KModes
# import pandera.typing as pdt

# k = 25

# intxn_cols = [c for c in analogs.columns if isinstance(c, tuple)]
# data_for_clustering = analogs.loc[analogs.outcome == 'acceptable'][intxn_cols].fillna(0)
# tallies = data_for_clustering.sum().to_dict()
# # probability scaled
# data_for_clustering = data_for_clustering.apply(lambda col: col / tallies[col.name],axis=0).fillna(0)

# #km = KModes(n_clusters=k, init='Huang', n_init=5, verbose=1)
# #labels = km.fit_predict(data_for_clustering)

# from scipy.cluster.vq import kmeans, vq

# centroid, variance = kmeans(data_for_clustering.values, k)
# labels, _ = vq(data_for_clustering.values, centroid)
# # list to series first for the correct indices:
# data_for_clustering['cluster_label']: pdt.Series[int] = labels
# analogs['cluster_label']: pdt.Series[float] = data_for_clustering.cluster_label
# analogs['cluster_label']: pdt.Series[int] = analogs['cluster_label'].fillna(-1).astype(int)

In [ ]:
from scipy.cluster.vq import kmeans, vq
from kmodes.kmodes import KModes
import pandera.typing as pdt
import enum

class clustering_algo(enum.Enum):
    KMEAN = 1
    KMODE = 2
    
class scaling_used(enum.Enum):
    NONE = 0
    CRUDE = 1
    PROB = 2
    FF = 3


# rubbish scaled
def rubbish_scale(col):
    """
    This is in effect inferior intermolecular LJ
    A hydrophobic is worth half, a salt bridge 2x
    
    :param col: 
    :return: 
    """
    if col.name[0] == 'hydroph_interaction':
        w = 0.5
    if col.name[0] in 'halogenbond':
        w = 1.5
    elif col.name[0] == 'saltbridge':
        w = 2.0
    else:
        w = 1.0
    return col * w

def force_score():
    """
    a potential between the relative atoms that is 
    the Coulombic LJ with cosine weight for angle as seen in the Dreiding H-bond term,
    https://docs.lammps.org/pair_hbond_dreiding.html#description
    """
    # determine which is the two atoms: maths mod of neighbourhood extraction?
    # extract the partial charge and atom type that Rosetta used
    raise NotImplemented

In [ ]:
# number of clusters sought
k = 10
scaling_mode = scaling_used.PROB

intxn_cols = [c for c in analogs.columns if isinstance(c, tuple)]
data_for_clustering = analogs.loc[analogs.outcome == 'acceptable'][intxn_cols].fillna(0).copy()
tallies = data_for_clustering.sum().to_dict()

if scaling_mode == scaling_used.PROB:
    # probability scaled
    data_for_clustering = data_for_clustering.apply(lambda col: col / tallies[col.name],axis=0).fillna(0)
    algo = clustering_algo.KMEAN
elif scaling_mode == scaling_used.CRUDE:
    # A hydrophobic is worth half, a salt bridge 2x
    data_for_clustering = data_for_clustering.apply(rubbish_scale, axis=0).fillna(0)
    algo = clustering_algo.KMEAN
elif scaling_mode == scaling_used.FF:
    raise NotImplemented
elif scaling_mode == scaling_used.NONE:
    # no scaling so Kmode
    algo = clustering_algo.KMODE
else:
    raise ValueError
    
if algo == clustering_algo.KMEAN:
    centroid, variance = kmeans(data_for_clustering.values, k)
    labels, _ = vq(data_for_clustering.values, centroid)
elif algo == clustering_algo.KMODE:
    km = KModes(n_clusters=k, init='Huang', n_init=5, verbose=1)
    labels = km.fit_predict(data_for_clustering)
else:
    raise ValueError

# list to series first for the correct indices:
data_for_clustering['cluster_label']: pdt.Series[int] = labels
analogs['cluster_label']: pdt.Series[float] = data_for_clustering.cluster_label
analogs['cluster_label']: pdt.Series[int] = analogs['cluster_label'].fillna(-1).astype(int)

In [ ]:
from collections import defaultdict
from Bio.SeqUtils import seq1
rank = defaultdict(int)

def r(c):
    rank[c] += 1
    return rank[c]

# 'cluster' is Similarity cluster
analogs = analogs.sort_values('full_penalty').drop_duplicates('cluster').reset_index(drop=True)
analogs['cluster_rank'] = analogs['cluster_label'].apply(r)
analogs = analogs.sort_values(['cluster_rank', 'full_penalty']).reset_index(drop=True).copy()

## Make uploadable SDF for Fragalysis

In [ ]:
from gist_import import GistImporter

# fu for fragalysis upload
fmodule = GistImporter.from_github('https://raw.githubusercontent.com/matteoferla/Fragment-hit-follow-up-chemistry/main/followup/prep_fragalysis.py')
prep = fmodule['prep']
generate_header = fmodule['generate_header']
floatify_columns = fmodule['floatify_columns']

In [ ]:
def narrate(row: pd.Series):
    grouped = defaultdict(list)
    for name, value in row.items():
        if not isinstance(name, tuple) or value == 0.:
            continue
        itxn_type, resn, resi = name
        grouped[itxn_type].append(seq1(resn, undef_code="X")+str(resi))
    narrative = f'cluster #{row.cluster_label} (rank {row.cluster_rank}); '
    for itxn_type in sorted(grouped):
        narrative += f'{itxn_type}:{"+".join(grouped[itxn_type])}; '
    return narrative

analogs['rationale'] = 'info ' + analogs.apply(narrate, axis=1)
assert analogs['rationale'].apply(len).max() < 255, 'Fix length!'

In [ ]:
def catalogize(name):
    if name[0] == 'Z':
        return name.split('-')[0]
    if name[0] == 'P':
        return 'PV-'+name.split('-')[1]
    else:
        return name

analogs['catalog_id'] = analogs.name.apply(catalogize)

In [ ]:
wanted_key_types = {'rationale': str, 
               'cluster_rank': int,
               'N_interactions': int, 
               'N_interactions_lost': int,
               '∆∆G': float, 
                    'comRMSD': float,
                    'N_rotatable_bonds': int, 
                     'N_unconstrained_atoms': int,
                    'N_constrained_atoms': int
                   }

for k, ktype in wanted_key_types.items():
    analogs[k] = analogs[k].astype(ktype)

wanted_keys = list(wanted_key_types)

def clean_names(names):
    deprefixed = [name.replace('A71EV2A-', '').split('S')[0] for name in names]
    return ','.join([name for name in deprefixed if name[0] == 'x'])

analogs['ref_mols'] = analogs.hit_names.apply(clean_names)

header: Chem.Mol = generate_header(method='A71-Fragmenstein',
                         ref_url='https://www.example.com',
                         submitter_name='Matteo Ferla',
                         submitter_email='matteo.ferla@stats.ox.ac.uk',
                         submitter_institution='University of Oxford',
                         extras=dict(zip(wanted_keys, wanted_keys))
                                  )
                                   
prep(analogs.head(200), 
     header, mol_col='minimized_mol', 
     name_col='catalog_id',
     outfile=f'A71_fragmenstein_clustered4.sdf',
     ref_pdb_name='x0554_0A',
     extras=wanted_keys
    )

## Junkyard

In [ ]:
from rdkit.Chem import Draw

Draw.MolsToGridImage(analogs.head(20).smiles.apply(Chem.MolFromSmiles))

## Junkyard

In [ ]:
# import numpy as np
# import numpy.typing as npt

# cluster_itxns = []
# for i in range(25):
#     wanted = list( *np.where(centroid[i] > 0) )
#     cluster_itxns.append( sorted([itxn for x, itxn in enumerate(intxn_cols) if x in wanted], key=operator.itemgetter(0)) )

In [ ]:
from collections import defaultdict
from Bio.SeqUtils import seq1

# def narrate(row: npt.ArrayLike[float], labels: List[Tuple[str, str, int]]):
#     wanted = list( *np.where(centroid_row > 0) )
#     itxns = [itxn for x, itxn in enumerate(labels) if x in wanted]
#     grouped = defaultdict(list)
#     for itxn_type, resn, resi in itxns:
#         grouped[itxn_type].append(seq1(resn, undef_code="X")+str(resi))
    
#     return 

In [ ]:
import pandas as pd
import numpy as np
import numpy.typing as npt
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.manifold import TSNE
import plotly.express as px

fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048) for mol in analogs.minimized_mol]
fp_array: npt.NDArray[bool] = np.array([list(fp) for fp in fps])

tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(fp_array)

# Create a DataFrame for the t-SNE results
tsne_df = pd.DataFrame(tsne_results, columns=['TSNE-1', 'TSNE-2'])

# Plot the t-SNE results using Plotly Express
fig = px.scatter(tsne_df, x='TSNE-1', y='TSNE-2', title='t-SNE Visualization of Compounds')
fig.show()